In [1]:
# importing library
import requests
import time
from bs4 import BeautifulSoup
import json
import pandas as pd
from pymongo import MongoClient

In [2]:
# Remove Character from the text
def Remove_Character(Text, characters):
    val = Text.get_text(separator=' ').rstrip().split()
    val = ' '.join(val)
    for c in characters:
        val = ''.join(val.split(c))
    return val

# Get the position value
def Get_Position_Value(Text_Value, value_address):
    val = Text_Value.get_text(separator=' ').rstrip().split()
    return(val[value_address])

# Join the array with given tag
def Join_Value_With(value, join_value):
    val = value.get_text(separator=' ').rstrip().split()
    val = join_value.join(val)
    return(val)

# To Reamove Unusal Spaces
def Remove_Unusal_Spaces(value):
    val = value.get_text(separator=' ').rstrip().split()
    val = ' '.join(val)
    return(val)

# Make the attributes naming 
def Name_Attribute(attribute):
    """remove the unusual spaces and convert the sentence into array"""
    attr = attribute.get_text(separator=' ').rstrip().split()
    attr = [x.casefold() for x in attr] # capitalize the first letter of the word
    attr = '_'.join(attr) # Join each word with "_"
    return(attr)

def Get_href(anchor):
    """scrap the anchor tag href"""
    href =  anchor.find('a', href=True)['href']
    return href

In [3]:
uni_search_url = 'https://www.usnews.com/best-colleges/search?schoolName='
uni_name = 'harvard university'
search_url = uni_search_url + uni_name

In [4]:
def scrapper(url):
    page = requests.get(url, timeout=1000, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.content,'html.parser')
    return soup, page.status_code

In [7]:
search_result_page, search_status = scrapper(search_url)
# print(search_result_page, search_status)

200


In [8]:
uni_link = search_result_page.find('a', class_="card-name", href=True)['href']
print(uni_link)

/best-colleges/harvard-university-2155


In [7]:
domain_name = 'https://www.usnews.com/'

In [8]:
uni_overview_url = domain_name + uni_link
uni_rankings_url = uni_overview_url + '/overall-rankings'
uni_admission_url = uni_overview_url + '/applying'
uni_academics_url  = uni_overview_url + '/academics'
uni_student_life_url  = uni_overview_url + '/student-life'
uni_user_review_url  = uni_overview_url + '/reviews'
uni_tution_and_fincance_url  = uni_overview_url + '/paying'
uni_campus_url  = uni_overview_url + '/campus-info'

In [9]:
print(uni_overview_url)
print(uni_rankings_url)
print(uni_admission_url)
print(uni_academics_url)
print(uni_student_life_url)
print(uni_user_review_url)
print(uni_tution_and_fincance_url)
print(uni_campus_url)

https://www.usnews.com//best-colleges/harvard-university-2155
https://www.usnews.com//best-colleges/harvard-university-2155/overall-rankings
https://www.usnews.com//best-colleges/harvard-university-2155/applying
https://www.usnews.com//best-colleges/harvard-university-2155/academics
https://www.usnews.com//best-colleges/harvard-university-2155/student-life
https://www.usnews.com//best-colleges/harvard-university-2155/reviews
https://www.usnews.com//best-colleges/harvard-university-2155/paying
https://www.usnews.com//best-colleges/harvard-university-2155/campus-info


In [10]:
# Data management
whole_data = {}

# Overview

In [11]:
# overview page
overview_section = {}
overview_page = scrapper(uni_overview_url)
# print(overview_page)

200


In [12]:
overview_intro = overview_page.find('div', class_='Raw-slyvem-0 util__RawContent-sc-1kd04gx-2 RClcr eNQGvA').p.text
general_info = overview_page.find_all('div', class_='Box-w0dun1-0 DataRow__Row-sc-1udybh3-0 knONpq keLhhz')
general_information = {}
for gi in general_info:
    paragraphs = gi.find_all('p')
    if len(paragraphs) == 2:
        general_information[Name_Attribute(paragraphs[0])] = paragraphs[1].text
    elif len(paragraphs) == 1:
        general_information[Name_Attribute(paragraphs[0])] = Get_href(gi)
print(general_information)

{'school_type': 'Private, Coed', 'year_founded': '1636', 'religious_affiliation': 'None', 'academic_calendar': 'Semester', 'setting': 'Urban', 'phone': '(617) 495-1000', 'school_website': 'http://www.harvard.edu/', 'median_starting_salary_of_alumni': '$69,000'}


In [13]:
overview_section['overview'] = overview_intro
overview_section['general_information'] = general_information

# Rankings

In [14]:
rankings_section = []
rankings_page = scrapper(uni_rankings_url)
# print(rankings_page)

200


In [15]:
rankings = rankings_page.find_all('a', class_='Anchor-byh49a-0 PlBer')
for r in rankings:
    rank = {}
    href = r['href']
    ranks = r.find_all('strong')
    if len(ranks) == 2:
        rank['url'] = r['href']
        rank['rank'] = ranks[0].text
        rank['title'] = ranks[1].text
        rankings_section.append(rank)

In [16]:
print(rankings_section)

[{'url': 'https://www.usnews.com/best-colleges/rankings/national-universities', 'rank': '#2', 'title': 'National Universities'}, {'url': 'https://www.usnews.com/best-colleges/rankings/national-universities/undergraduate-teaching', 'rank': '#13', 'title': 'Best Undergraduate Teaching'}, {'url': 'https://www.usnews.com/best-colleges/rankings/national-universities/best-value', 'rank': '#3', 'title': 'Best Value Schools'}, {'url': 'https://www.usnews.com/best-colleges/rankings/national-universities/innovative', 'rank': '#23', 'title': 'Most Innovative Schools'}, {'url': 'https://www.usnews.com/best-colleges/rankings/national-universities/social-mobility', 'rank': '#211', 'title': 'Top Performers on Social Mobility'}, {'url': 'https://www.usnews.com/best-colleges/rankings/engineering-doctorate', 'rank': '#27', 'title': 'Best Undergraduate Engineering Programs'}, {'url': 'https://www.usnews.com/best-colleges/rankings/computer-science/biocomputing-bioinformatics-biotechnology', 'rank': '#10',

# Admission

In [17]:
admission_section = {}
admission_page = scrapper(uni_admission_url)
# print(admission_page)

200


In [18]:
summary_box = admission_page.find('div', class_='summary-box')
summary = summary_box.find_all('p')
paragraph = ''
for p in summary:
    paragraph = paragraph + Remove_Unusal_Spaces(p)
print(paragraph)

Harvard University admissions is most selective with an acceptance rate of 5% and an early acceptance rate of 13.9%. Half the applicants admitted to Harvard University have an SAT score between 1460 and 1580 or an ACT score of 33 and 35. However, one quarter of admitted applicants achieved scores above these ranges and one quarter scored below these ranges. The application deadline is Jan. 1 and the application fee at Harvard University is $75. Admissions officials at Harvard University consider a student's GPA an academic factor. An applicant's high school class rank is not considered but letters of recommendation are considered by admissions officials at Harvard University. To see additional academic factors along with other school data, learn more about College Compass .Harvard University admissions is most selective with an acceptance rate of 5% and an early acceptance rate of 13.9%. Half the applicants admitted to Harvard University have an SAT score between 1460 and 1580 or an AC

In [19]:
# admissions_stats = {}
# admission_stat = admission_page.find('div', class_='quick-stat-box')
# stat = admission_stat.find_all('div')

In [20]:
# for s in stat:
#     admissions_stats[Name_Attribute(s.dl.dt)] = s.dl.dd.text
# print(admissions_stats)

In [21]:
admission_info = admission_page.find_all('div', class_='Box-w0dun1-0 DataRow__Row-sc-1udybh3-0 knONpq fPkWrz Display__DataRowBox-h3gn08-1 kwTiaI')
admission_information = {}
for info in admission_info:
    para = info.find_all('p')
    if para[0].text != 'UNLOCK WITH COMPASS':
        admission_information[Name_Attribute(para[0])] = para[1].text
print(admission_information)

{'admission_interview': 'Recommended', 'required_standardized_tests': 'Either SAT or ACT', 'sat/act_scores_must_be_received_by': 'March 6', 'application_deadline': 'Jan. 1', 'early_decision_deadline': 'N/A', 'application_fee': '$75', 'common_application_accepted': 'Yes', 'deadline_for_housing_deposit': 'N/A', 'selectivity': 'Most selective', 'acceptance_rate': '5%', 'early_decision_acceptance_rate': 'N/A', 'sats_on_1600_scale': 'N/A', 'accepting_applications': 'Fall', 'minimum_credits_to_apply': '16', 'minimum_required_college_gpa': 'N/A', 'has_guaranteed_admission_agreement_with_at_least_one_other_institution': 'N/A', 'guaranteed_admission_agreement_url': 'N/A', 'separate_application_form_required': 'No', 'conditional_admission_offered': 'No', 'early_decision_or_early_action_options_available': 'Early action only', 'toefl_(paper)': 'Considered', 'toefl_(internet-based)': 'Considered', 'michigan_test': 'N/A', 'ielts': 'N/A', 'sat': 'Required', 'act': 'Required', 'advance_deposit_requir

In [22]:
admission_contact_section = admission_page.find('div', class_='Box-w0dun1-0 DataRow__Row-sc-1udybh3-0 knONpq fPkWrz Display__DataRowBox-h3gn08-1 kwTiaI datarow-list')
contact_section = admission_contact_section.find_all('div')
admission_contact_info = {}
admission_contact_info['name'] = Remove_Unusal_Spaces(contact_section[0])
admission_contact_info['office'] = Remove_Unusal_Spaces(contact_section[1])
admission_contact_info['phone_number'] = Remove_Unusal_Spaces(contact_section[2])
admission_contact_info['email'] = Remove_Unusal_Spaces(contact_section[3])
print(admission_contact_info)

{'name': 'Judy Partington', 'office': 'International Admissions Administrator', 'phone_number': '(617) 495-1551', 'email': 'intladm@fas.harvard.edu'}


In [23]:
admission_section['summary'] = paragraph
admission_section['admission_information'] = admission_information
admission_section['admission_contact_information'] = admission_contact_info
# admission_section['admissions_stats'] = admissions_stats

In [24]:
print(admission_section)

{'summary': "Harvard University admissions is most selective with an acceptance rate of 5% and an early acceptance rate of 13.9%. Half the applicants admitted to Harvard University have an SAT score between 1460 and 1580 or an ACT score of 33 and 35. However, one quarter of admitted applicants achieved scores above these ranges and one quarter scored below these ranges. The application deadline is Jan. 1 and the application fee at Harvard University is $75. Admissions officials at Harvard University consider a student's GPA an academic factor. An applicant's high school class rank is not considered but letters of recommendation are considered by admissions officials at Harvard University. To see additional academic factors along with other school data, learn more about College Compass .Harvard University admissions is most selective with an acceptance rate of 5% and an early acceptance rate of 13.9%. Half the applicants admitted to Harvard University have an SAT score between 1460 and 

# Academics

In [25]:
academics_section = {}
academics_page = scrapper(uni_academics_url)
# print(academics_page)

200


In [26]:
academic_summary_box = academics_page.find('div', class_='summary-box')
academic_summary = academic_summary_box.find_all('p')
academic_paragraph = ''
for p in academic_summary:
    academic_paragraph = academic_paragraph + Remove_Unusal_Spaces(p)
print(academic_paragraph)

The student-faculty ratio at Harvard University is 5:1, and the school has 76.3% of its classes with fewer than 20 students. The most popular majors at Harvard University include: Social Sciences, General; Biology/Biological Sciences, General; Mathematics, General; Computer and Information Sciences, General; History, General; Physical Sciences, General; Engineering, General; Psychology, General; English Language and Literature, General; and Visual and Performing Arts, General. The average freshman retention rate, an indicator of student satisfaction, is 92%.


In [27]:
most_popular_majors = []
popular_majors = academics_page.find('div', class_='Box-w0dun1-0 DataRow__Row-sc-1udybh3-0 knONpq fPkWrz Display__DataRowBox-h3gn08-1 kwTiaI datarow-table truncated')
pop_majors = popular_majors.find_all('li')
for m in pop_majors:
    para = m.find_all('p')
    if len(para) == 2:
        most_popular_majors.append({
            'name': para[0].text,
            'percentage': para[1].text
        })
print(most_popular_majors)

[{'name': 'Social Sciences, General', 'percentage': ' 27%'}, {'name': 'Biology/Biological Sciences, General', 'percentage': ' 12%'}, {'name': 'Mathematics, General', 'percentage': ' 12%'}, {'name': 'Computer and Information Sciences, General', 'percentage': ' 11%'}, {'name': 'History, General', 'percentage': ' 9%'}, {'name': 'Physical Sciences, General', 'percentage': ' 7%'}, {'name': 'Engineering, General', 'percentage': ' 5%'}, {'name': 'Psychology, General', 'percentage': ' 4%'}, {'name': 'English Language and Literature, General', 'percentage': ' 3%'}, {'name': 'Visual and Performing Arts, General', 'percentage': ' 3%'}]


In [28]:
academics_section['academic_summary'] = academic_paragraph
academics_section['popular_majors'] = most_popular_majors

# Student Life

In [29]:
student_life_section = {}
student_life_page = scrapper(uni_student_life_url)
# print(student_life_page)

200


In [30]:
student_life_summary_box = student_life_page.find('div', class_='summary-box')
student_life_summary = student_life_summary_box.find_all('p')
student_life_paragraph = ''
for p in student_life_summary:
    student_life_paragraph = student_life_paragraph + Remove_Unusal_Spaces(p)
print(student_life_paragraph)

Harvard University has a total undergraduate enrollment of 5,222 (fall 2020), with a gender distribution of 49% male students and 51% female students. In sports, Harvard University is part of the NCAA I.


In [31]:
uni_student_body = []
student_body = student_life_page.find_all('div', class_='Box-w0dun1-0 DataRow__Row-sc-1udybh3-0')
print(student_body)
for stud in student_body:
    para = stud.find_all('p')
    if (len(para) == 2) & (para[0].text != 'UNLOCK WITH COMPASS'):
        uni_student_body.append({
            'title': para[0].text,
            'value': para[1].text,
        })
print(uni_student_body)

[]
[]


In [32]:
student_life_section['student_life_summary'] = student_life_paragraph
student_life_section['student_life_information'] = uni_student_body

# Tuition and Financial

In [33]:
tuition_and_financial_section = {}
tuition_and_financial_page = scrapper(uni_tution_and_fincance_url)
# print(tuition_and_financial_page)

200


In [34]:
tuition_and_financial_summary_box = tuition_and_financial_page.find('div', class_='summary-box')
tuition_and_financial_summary = tuition_and_financial_summary_box.find_all('p')
tuition_and_financial_paragraph = ''
for p in tuition_and_financial_summary:
    tuition_and_financial_paragraph = tuition_and_financial_paragraph + Remove_Unusal_Spaces(p)
print(tuition_and_financial_paragraph)

Harvard University's tuition is $55,587. Compared with the national average cost of tuition of $41,281, Harvard University is more expensive. These figures include both tuition and fees, also referred to as the sticker price . Fees differ by institution and may fund library services, student gym facilities, student centers, technology resources and campus health centers. As you’re comparing costs of different institutions, also consider the total cost and the net price. The total cost is the sticker price, plus the cost of room and board, books and supplies, and transportation and personal expenses. At Harvard University, the total cost is $75,891. The net price is the average cost of the university after aid and scholarship funds are discounted from the total cost, which comes in at $20,023 for the average student receiving need-based aid.Harvard University's tuition is $55,587. Compared with the national average cost of tuition of $41,281, Harvard University is more expensive.These f

In [35]:
financial_info = []
finance = tuition_and_financial_page.find_all('div', class_='Box-w0dun1-0 DataRow__Row-sc-1udybh3-0 knONpq fPkWrz Display__DataRowBox-h3gn08-1 kwTiaI')
for f in finance:
    para = f.find_all('p')
    if (len(para) == 2) & (para[0].text != 'UNLOCK WITH COMPASS'):
        financial_info.append({
            'titel': para[0].text,
            'value': para[1].text,
        })
print(financial_info)

[{'titel': 'Tuition and fees', 'value': '$55,587(2021-22)'}, {'titel': 'Room and board', 'value': '$18,941(2021-22)'}, {'titel': 'Institution is G.I. Bill certified', 'value': 'Yes'}, {'titel': 'Institution participates in Yellow Ribbon Program', 'value': 'Yes'}, {'titel': 'Average total indebtedness of 2020 graduating class', 'value': '$18,197'}, {'titel': 'Graduating students who have borrowed (any loan type, 2020)', 'value': '20%'}]


In [36]:
tuition_and_financial_section['summary'] = tuition_and_financial_paragraph
tuition_and_financial_section['financial_information'] = financial_info

# Final Data

In [37]:
whole_data['overview_section'] = overview_section
whole_data['rankings'] = rankings_section
whole_data['admission'] = admission_section
whole_data['academics'] = academics_section
whole_data['student_life'] = student_life_section
whole_data['tuition_and_financial'] = tuition_and_financial_section

In [38]:
print(whole_data)

{'overview_section': {'overview': "Harvard University is a private institution that was founded in 1636. It has a total undergraduate enrollment of 5,222 (fall 2020), its setting is urban, and the campus size is 5,076 acres. It utilizes a semester-based academic calendar. Harvard University's ranking in the 2022 edition of Best Colleges is National Universities, #2. Its tuition and fees are $55,587.", 'general_information': {'school_type': 'Private, Coed', 'year_founded': '1636', 'religious_affiliation': 'None', 'academic_calendar': 'Semester', 'setting': 'Urban', 'phone': '(617) 495-1000', 'school_website': 'http://www.harvard.edu/', 'median_starting_salary_of_alumni': '$69,000'}}, 'rankings': [{'url': 'https://www.usnews.com/best-colleges/rankings/national-universities', 'rank': '#2', 'title': 'National Universities'}, {'url': 'https://www.usnews.com/best-colleges/rankings/national-universities/undergraduate-teaching', 'rank': '#13', 'title': 'Best Undergraduate Teaching'}, {'url': '